## R Magic

In [1]:
# activate R magic
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


## Type conversions

In [2]:
%cd /content/drive/My Drive/Colab Notebooks/AI_relative/
!ls
%cd Data_mining_in_R/
!ls

/content/drive/My Drive/Colab Notebooks/AI_relative
algorithm_explore  deep_learning_GAN  NLP_tutorial	 pytorch_study
Data_mining_in_R   GAN_sad.ipynb      push_to_git.ipynb
/content/drive/My Drive/Colab Notebooks/AI_relative/Data_mining_in_R
dirty_iris_check.txt	tutorial_check_and_imputation.ipynb
dirty_iris_correct.txt	warpbreaks.csv


In [3]:
%%R
warpbreaks.df = read.csv("warpbreaks.csv")

find columns type

In [5]:
%%R
str(warpbreaks)

'data.frame':	54 obs. of  3 variables:
 $ breaks : num  26 30 54 25 70 52 51 26 67 18 ...
 $ wool   : Factor w/ 2 levels "A","B": 1 1 1 1 1 1 1 1 1 1 ...
 $ tension: Factor w/ 3 levels "L","M","H": 1 1 1 1 1 1 1 1 1 2 ...


convert numeric to integer

In [7]:
%%R
breaks = warpbreaks["breaks"]
is.list(breaks)
is.recursive(breaks)
breaks_int = as.integer(unlist(breaks))
str(breaks_int)

 int [1:54] 26 30 54 25 70 52 51 26 67 18 ...


check type of function

In [9]:
%%R
typeof(mean)

[1] "closure"


### factor to character and numeric and int

In [11]:
%%R
v <- factor(c("2", "3", "5", "7", "11"))
v
vChar <- as.character(v)
vChar

[1] "2"  "3"  "5"  "7"  "11"


In [13]:
%%R
vNum <- as.numeric(v)
vNum

[1] 2 3 4 5 1


In [14]:
%%R
vInt <- as.integer(v)
vInt

[1] 2 3 4 5 1


## Read

In [22]:
%cd /content/drive/My Drive/Colab Notebooks/AI_relative/
!ls
%cd Data_mining_in_R/
!ls

/content/drive/My Drive/Colab Notebooks/AI_relative
algorithm_explore  deep_learning_GAN  NLP_tutorial	 pytorch_study
Data_mining_in_R   GAN_sad.ipynb      push_to_git.ipynb
/content/drive/My Drive/Colab Notebooks/AI_relative/Data_mining_in_R
dirty_iris_check.txt	example.txt			     warpbreaks.csv
dirty_iris_correct.txt	tutorial_check_and_imputation.ipynb


In [47]:
%%R
example <- readLines("example.txt")
example

[1] "// Survey data. Created : 21 May 2013"
[2] "// Field 1: Gender"                   
[3] "// Field 2: Age (in years)"           
[4] "// Field 3: Weight (in kg)"           
[5] "M;28;81.3"                            
[6] "male;45;"                             
[7] "Female;17;57,2"                       
[8] "fem.;64;62.8"                         


### regular expression in grepl

In [48]:
%%R
pos <- grepl("^//", example)
comments <- example[pos]
data <- example[!pos]
data

[1] "M;28;81.3"      "male;45;"       "Female;17;57,2" "fem.;64;62.8"  


In [49]:
%%R
comments

[1] "// Survey data. Created : 21 May 2013"
[2] "// Field 1: Gender"                   
[3] "// Field 2: Age (in years)"           
[4] "// Field 3: Weight (in kg)"           


extract data from the first comment line

In [50]:
%%R
firstComment <- comments[1]
firstData <- data[1]
typeof(data[1])

[1] "character"


handle with data

In [51]:
%%R
data_list <- strsplit(data, ";") # get a list of character vectors
assignFields <- function(x){
  out <- character(3)
  out[1] <- ifelse(grepl("^(M|m)", x[1]), "male", "female")
  out[2] <- as.numeric(x[2])
  out[3] <- ifelse(length(x[3])>0, x[3], 'NA')
  out
}
data_processed <- lapply(data_list, assignFields)
dataMtr <- matrix(unlist(data_processed), ncol = 3, byrow = TRUE)

In [52]:
%%R
dataMtr

     [,1]     [,2] [,3]  
[1,] "male"   "28" "81.3"
[2,] "male"   "45" NA    
[3,] "female" "17" "57,2"
[4,] "female" "64" "62.8"


### add columns name

In [53]:
%%R
colnames(dataMtr) <- c("gender", "Age (in years)", "Weight (in kg)")
real_example <- as.data.frame(dataMtr, stringsAsFactors = FALSE)
real_example

  gender Age (in years) Weight (in kg)
1   male             28           81.3
2   male             45           <NA>
3 female             17           57,2
4 female             64           62.8


### coerce to dataframe

In [54]:
%%R
data.frame(real_example)

  gender Age..in.years. Weight..in.kg.
1   male             28           81.3
2   male             45           <NA>
3 female             17           57,2
4 female             64           62.8


In [55]:
%%R
str(real_example)

'data.frame':	4 obs. of  3 variables:
 $ gender        : chr  "male" "male" "female" "female"
 $ Age (in years): chr  "28" "45" "17" "64"
 $ Weight (in kg): chr  "81.3" NA "57,2" "62.8"


### factorize a vector use adist

In [56]:
%%R
codes <- c("male", "female")
D <- adist(real_example$gender, codes)
D

     [,1] [,2]
[1,]    0    2
[2,]    0    2
[3,]    2    0
[4,]    2    0


In [57]:
%%R
i <- apply(D, 1, which.min)
i

[1] 1 1 2 2


add label

In [58]:
%%R
gender = c("man", "woman")
gender[i]

[1] "man"   "man"   "woman" "woman"


In [59]:
%%R
real_example$gender <- as.factor(gender[i])

In [60]:
%%R
str(real_example)

'data.frame':	4 obs. of  3 variables:
 $ gender        : Factor w/ 2 levels "man","woman": 1 1 2 2
 $ Age (in years): chr  "28" "45" "17" "64"
 $ Weight (in kg): chr  "81.3" NA "57,2" "62.8"


### char -> int

In [61]:
%%R
real_example$"Age (in years)" <- as.integer(real_example$"Age (in years)")
real_example

  gender Age (in years) Weight (in kg)
1    man             28           81.3
2    man             45           <NA>
3  woman             17           57,2
4  woman             64           62.8


In [63]:
%%R
str(real_example)

'data.frame':	4 obs. of  3 variables:
 $ gender        : Factor w/ 2 levels "man","woman": 1 1 2 2
 $ Age (in years): int  28 45 17 64
 $ Weight (in kg): chr  "81.3" NA "57,2" "62.8"


### char -> numeric

In [68]:
%%R
real_example$"Weight (in kg)" <- as.numeric(gsub(",", ".", real_example$"Weight (in kg)"))
real_example

  gender Age (in years) Weight (in kg)
1    man             28           81.3
2    man             45             NA
3  woman             17           57.2
4  woman             64           62.8


In [70]:
%%R
str(real_example)

'data.frame':	4 obs. of  3 variables:
 $ gender        : Factor w/ 2 levels "man","woman": 1 1 2 2
 $ Age (in years): int  28 45 17 64
 $ Weight (in kg): num  81.3 NA 57.2 62.8
